In [1]:
# Configuration

# days used in calculating ratios
DAYS_RATIO = [3, 5, 10, 15, 30, 60, 120, 250]

# risk free rate
RISK_FREE_RATE = 0.04

In [2]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [3]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")
# print(f"DOWNLOAD_DIR: {DOWNLOAD_DIR}")
print(f"DEST_DIR: {DEST_DIR}\n")

# Build paths
source_path = Path(DEST_DIR) / f'{date_str}_df_OHLCV_clean.parquet'
dest_path = Path(DEST_DIR) / f'{date_str}_df_perf_ratios.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

date_str: 2025-04-08
DEST_DIR: ..\data

source_path: ..\data\2025-04-08_df_OHLCV_clean.parquet
dest_path: ..\data\2025-04-08_df_perf_ratios.parquet


In [4]:
import pandas as pd

# Get tickers from df_finviz.pkl file
df_finviz = pd.read_parquet(f'..\data\{date_str}_df_finviz.parquet')
tickers = df_finviz.index.to_list()

In [5]:
import pandas as pd

pd.set_option('display.width', 1000) 

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# # Display the first few rows of the DataFrame to verify
# df = df[['Adj Close']].copy()
display(df)

Open   High    Low  Close  Adj Close   Volume   Adj Open   Adj High    Adj Low
Symbol Date                                                                                       
ILMN   2025-04-08  75.76  75.54  69.38  70.30      70.30  3212285  75.760000  75.540000  69.380000
       2025-04-07  70.61  75.83  69.55  73.45      73.45  5170600  70.610000  75.830000  69.550000
       2025-04-04  75.40  75.43  70.64  74.16      74.16  3913400  75.400000  75.430000  70.640000
       2025-04-03  79.51  79.97  76.26  76.42      76.42  2117600  79.510000  79.970000  76.260000
       2025-04-02  78.27  82.76  77.99  81.88      81.88  1908300  78.270000  82.760000  77.990000
...                  ...    ...    ...    ...        ...      ...        ...        ...        ...
RF     2024-04-15  19.38  19.65  19.09  19.22      18.41  6936600  18.563257  18.821878  18.285479
       2024-04-12  19.05  19.30  18.94  19.19      18.38  6276900  18.245909  18.485357  18.140552
       2024-04-11  19.72  19.76  19.20  19.41      18.59  8061100  18.886904  18.925214  18.388872
       2024-04-10  20.29  20.30  19.54  19.68      18.85  6684200  19.434273  19.443852  18.715904
       2024-04-09  20.50  20.69  20.36  20.69      19.82  5321400  19.637989  19.820000  19.503876

[381520 rows x 9 columns]

In [6]:
import pandas as pd

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# Display the first few rows of the DataFrame to verify
df = df[['Adj Close']].copy()
display(df)

Adj Close
Symbol Date                 
ILMN   2025-04-08      70.30
       2025-04-07      73.45
       2025-04-04      74.16
       2025-04-03      76.42
       2025-04-02      81.88
...                      ...
RF     2024-04-15      18.41
       2024-04-12      18.38
       2024-04-11      18.59
       2024-04-10      18.85
       2024-04-09      19.82

[381520 rows x 1 columns]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 381520 entries, ('ILMN', Timestamp('2025-04-08 00:00:00')) to ('RF', Timestamp('2024-04-09 00:00:00'))
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Close  381520 non-null  float64
dtypes: float64(1)
memory usage: 4.4+ MB


In [8]:
import pandas as pd

def select_tickers_data(df, tickers):
  """
  Selects data for a list of tickers from a DataFrame with a MultiIndex
  where the first level is the ticker and the second level is the date.

  Args:
    df (pd.DataFrame): The input DataFrame with a MultiIndex.
    tickers (list): A list of ticker symbols to select.

  Returns:
    pd.DataFrame: A DataFrame containing only the data for the specified tickers.
                  Rows for tickers not found will not be included.
  """
  valid_tickers = [t for t in tickers if t in df.index.get_level_values(0)]  # Filter out the tickers that do not exist

  if not valid_tickers:
    print("No valid tickers found in the DataFrame. Returning an empty DataFrame.")
    return pd.DataFrame()

  try:
    ticker_df = df.loc[valid_tickers]
    return ticker_df
  except KeyError as e:
    print(f"KeyError after filtering valid tickers: {e}") #Added to help debugging.
    return pd.DataFrame()


# Example usage:
# ticker_list = ["UBS", "AAPL", "MSFT", "GEV"]  # Include some valid and invalid tickers
selected_data = select_tickers_data(df, tickers)
print(selected_data)

                   Adj Close
Symbol Date                 
MSFT   2025-04-08     354.56
       2025-04-07     357.86
       2025-04-04     359.84
       2025-04-03     373.11
       2025-04-02     382.14
...                      ...
IYH    2024-04-15      57.52
       2024-04-12      57.72
       2024-04-11      58.53
       2024-04-10      58.78
       2024-04-09      59.52

[381520 rows x 1 columns]


In [9]:
# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

import sys

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

import utils

list_dfs = utils.get_latest_dfs(selected_data, DAYS_RATIO)

In [10]:
import numpy as np

all_results = {}

for _df in list_dfs:
    tickers_in_df = _df.index.get_level_values(0).unique()
    for ticker in tickers_in_df:
        # Suppress division warnings just for this calculation
        with np.errstate(divide='ignore', invalid='ignore'):
            result_df = utils.analyze_stock(_df, ticker, risk_free_rate=RISK_FREE_RATE)
        
        if result_df is not None:
            # Extract the ticker name from the result_df index
            ticker_name = result_df.index[0]
            # Convert the single-row DataFrame to a dictionary
            metrics = result_df.iloc[0].to_dict()
            
            # Update the existing ticker entry or create a new one
            if ticker_name in all_results:
                all_results[ticker_name].update(metrics)
            else:
                all_results[ticker_name] = metrics

if all_results:
    combined_df = pd.DataFrame.from_dict(all_results, orient='index')
    print("\nCombined performance metrics DataFrame:")
    print(combined_df)
else:
    print("No performance metrics were calculated.")


Combined performance metrics DataFrame:
      Sharpe 3d  Sortino 3d  Omega 3d  Sharpe 5d  Sortino 5d  Omega 5d  Sharpe 10d  Sortino 10d  Omega 10d  Sharpe 15d  Sortino 15d  Omega 15d  Sharpe 30d  Sortino 30d  Omega 30d  Sharpe 60d  Sortino 60d  Omega 60d  Sharpe 120d  Sortino 120d  Omega 120d  Sharpe 250d  Sortino 250d  Omega 250d
A     -8.578090   -9.724640  0.133660 -20.008903  -13.107840  0.035408  -11.769157    -9.978208   0.119172   -9.661497    -8.661607   0.159365   -6.955251    -6.948443   0.291651   -4.855183    -5.326868   0.423723    -2.725345     -3.339190    0.626001    -1.336009     -1.708551    0.791007
AA   -13.990207  -12.381732  0.000000 -25.933875  -14.025664  0.000000  -14.362839   -11.082723   0.058044  -12.372732   -10.070277   0.065104   -5.089000    -5.450813   0.417912   -3.418318    -3.872694   0.556891    -2.385336     -2.861628    0.660848    -0.785485     -1.051124    0.877581
AAL   -4.838442   -6.762121  0.397582 -10.084630   -9.605393  0.151291   -9.4912

In [11]:
import numpy as np
import pandas as pd

def find_nan_inf(df):
    """
    Finds locations of NaN, Inf, and -Inf values in a DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame to check
        
    Returns:
        dict: Dictionary with column names as keys and dicts of 
              {type: indices} as values
    """
    nan_inf_locations = {}
    for col in df.columns:
        locations = {
            'NaN': df[col].index[df[col].isnull()].tolist(),
            'Inf': df[col].index[df[col] == np.inf].tolist(),
            '-Inf': df[col].index[df[col] == -np.inf].tolist()
        }
        if any(locations.values()):
            nan_inf_locations[col] = locations
    return nan_inf_locations

def print_nan_inf_locations(locations, message):
    """Prints NaN/Inf locations with a descriptive message."""
    print(message)
    if locations:
        for col, locs in locations.items():
            print(f"Column: {col}")
            for typ, indices in locs.items():
                if indices:
                    print(f"  {typ}: {indices}")
    else:
        print("No NaN or Inf values found.")

def get_column_replacement_values(df):
    """
    Calculate replacement values for each column (max for Inf/NaN, min for -Inf).
    
    Args:
        df (pd.DataFrame): Input DataFrame
        
    Returns:
        tuple: (max_values, min_values) dictionaries with column replacement values
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    max_values = {}
    min_values = {}
    
    for col in numeric_cols:
        # Get finite values only
        finite_vals = df[col][np.isfinite(df[col])]
        if len(finite_vals) > 0:
            max_values[col] = finite_vals.max()
            min_values[col] = finite_vals.min()
        else:
            # If no finite values, use default
            max_values[col] = 1e5
            min_values[col] = -1e5
    
    return max_values, min_values

# Check for NaN or Inf values
has_nan_inf = combined_df.isnull().any().any() or not np.isfinite(combined_df).all().all()
print(f'combined_df has_nan_inf: {has_nan_inf}')

if has_nan_inf:
    # Calculate replacement values for each column
    max_values, min_values = get_column_replacement_values(combined_df)
    print("Replacing NaN/Inf values with column-specific max/min values")
    
    nan_inf_locations_before = find_nan_inf(combined_df)
    print_nan_inf_locations(nan_inf_locations_before, "NaN/Inf Locations BEFORE Replacement:")
    
    # Create cleaned DataFrame
    df_to_save = combined_df.copy()
    
    # Replace values column by column
    for col in df_to_save.columns:
        if col in max_values:  # Only numeric columns will be in max_values
            # Replace NaN and +Inf with max value
            df_to_save[col] = df_to_save[col].fillna(max_values[col])
            df_to_save[col] = df_to_save[col].replace([np.inf], max_values[col])
            # Replace -Inf with min value
            df_to_save[col] = df_to_save[col].replace([-np.inf], min_values[col])
else:
    print("No NaN or Inf values found.")
    df_to_save = combined_df

# Verification
print(f'df_to_save has_nan_inf: {df_to_save.isnull().any().any() or not np.isfinite(df_to_save).all().all()}')
print_nan_inf_locations(find_nan_inf(df_to_save), "NaN/Inf Locations AFTER Replacement:")

# Output cleaned DataFrame
print(df_to_save)

combined_df has_nan_inf: True
Replacing NaN/Inf values with column-specific max/min values
NaN/Inf Locations BEFORE Replacement:
Column: Sharpe 3d
  -Inf: ['BSCP', 'TBIL']
Column: Sortino 3d
  Inf: ['AEM', 'ANET', 'APP', 'AVGO', 'AXON', 'BA', 'BWXT', 'CACC', 'CEG', 'CRWD', 'CVNA', 'CVS', 'CW', 'DRS', 'DTM', 'EME', 'EQT', 'FHN', 'FIX', 'FTNT', 'GE', 'GEV', 'HUBB', 'HUM', 'ICL', 'IOT', 'ITA', 'JPM', 'LNG', 'MFG', 'NEM', 'NFLX', 'NU', 'PAAA', 'PPA', 'PSTG', 'RCL', 'SNOW', 'SRAD', 'TKC', 'UI', 'WDC']
Column: Omega 3d
  NaN: ['AEM', 'ANET', 'APP', 'AVGO', 'AXON', 'BA', 'BWXT', 'CACC', 'CEG', 'CRWD', 'CVNA', 'CVS', 'CW', 'DRS', 'DTM', 'EME', 'EQT', 'FHN', 'FIX', 'FTNT', 'GE', 'GEV', 'HUBB', 'HUM', 'ICL', 'IOT', 'ITA', 'JPM', 'LNG', 'MFG', 'NEM', 'NFLX', 'NU', 'PAAA', 'PPA', 'PSTG', 'RCL', 'SNOW', 'SRAD', 'TKC', 'UI', 'WDC']
df_to_save has_nan_inf: False
NaN/Inf Locations AFTER Replacement:
No NaN or Inf values found.
      Sharpe 3d  Sortino 3d  Omega 3d  Sharpe 5d  Sortino 5d  Omega 5d  Sha

In [12]:
ticker_w_inf = nan_inf_locations_before['Sortino 3d']['Inf'][0]
ticker_w_inf 

'AEM'

#### Use Cell below to check the inf, -inf and NaN replacement

In [13]:
print(f"combined_df.loc['ticker_w_inf']")
display(combined_df.loc[ticker_w_inf])
print(f"\ndf_to_save.loc['ticker_w_inf']")
display(df_to_save.loc[ticker_w_inf])

combined_df.loc['ticker_w_inf']


Sharpe 3d       19.852088
Sortino 3d            inf
Omega 3d              NaN
Sharpe 5d       -6.125631
Sortino 5d      -6.508496
Omega 5d         0.180007
Sharpe 10d      -2.384339
Sortino 10d     -2.678471
Omega 10d        0.559882
Sharpe 15d      -2.258406
Sortino 15d     -2.504830
Omega 15d        0.568917
Sharpe 30d       0.501304
Sortino 30d      0.633759
Omega 30d        1.105608
Sharpe 60d       2.598280
Sortino 60d      3.658088
Omega 60d        1.609983
Sharpe 120d      1.297042
Sortino 120d     1.855690
Omega 120d       1.252640
Sharpe 250d      1.672172
Sortino 250d     2.378832
Omega 250d       1.326936
Name: AEM, dtype: float64


df_to_save.loc['ticker_w_inf']


Sharpe 3d        19.852088
Sortino 3d      722.242451
Omega 3d         65.342471
Sharpe 5d        -6.125631
Sortino 5d       -6.508496
Omega 5d          0.180007
Sharpe 10d       -2.384339
Sortino 10d      -2.678471
Omega 10d         0.559882
Sharpe 15d       -2.258406
Sortino 15d      -2.504830
Omega 15d         0.568917
Sharpe 30d        0.501304
Sortino 30d       0.633759
Omega 30d         1.105608
Sharpe 60d        2.598280
Sortino 60d       3.658088
Omega 60d         1.609983
Sharpe 120d       1.297042
Sortino 120d      1.855690
Omega 120d        1.252640
Sharpe 250d       1.672172
Sortino 250d      2.378832
Omega 250d        1.326936
Name: AEM, dtype: float64

In [14]:
# Save the DataFrame to parquet file (always do this)
df_to_save.to_parquet(dest_path, engine='pyarrow', compression='zstd')
print(f"DataFrame successfully saved to {dest_path}")



DataFrame successfully saved to ..\data\2025-04-08_df_perf_ratios.parquet


In [15]:
import pandas as pd
import numpy as np
from tabulate import tabulate

_symbols = ['USFR','AAPL', 'MSFT', 'GOOG', 'NVDA', 'IBIT', 'GLD', 'VCIT']
# Create and display the formatted table
selected_df = df_to_save.loc[_symbols] #Use df_to_save here
print(tabulate(selected_df, headers='keys', tablefmt='grid', floatfmt='.4f'))

+------+-------------+--------------+------------+-------------+--------------+------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+---------------+----------------+--------------+---------------+----------------+--------------+
|      |   Sharpe 3d |   Sortino 3d |   Omega 3d |   Sharpe 5d |   Sortino 5d |   Omega 5d |   Sharpe 10d |   Sortino 10d |   Omega 10d |   Sharpe 15d |   Sortino 15d |   Omega 15d |   Sharpe 30d |   Sortino 30d |   Omega 30d |   Sharpe 60d |   Sortino 60d |   Omega 60d |   Sharpe 120d |   Sortino 120d |   Omega 120d |   Sharpe 250d |   Sortino 250d |   Omega 250d |
+======+=============+==============+============+=============+==============+============+==============+===============+=============+==============+===============+=============+==============+===============+=============+==============+===============+====